In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
%cd .. 
%cd ..

c:\Users\arik_\Documents\Dokumente\Job_Clausthal\TNTM\TNTM_Revision_TNNLS


In [2]:
import torch
import pickle


In [6]:
from TNTM.Benchmark.TopMost2OctisAdapter import TopMost2OctisAdapter

from topmost.models import ProdLDA

In [7]:


#with open("C:\\Users\\arik_\\\Documents\\Dokumente\\Job_Clausthal\\TNTM\\TNTM_Revision_TNNLS\\TNTM\\Data\\DataOctis2\\octis_dataset_20ng.pickle", "rb") as file:
#    bow_data = pickle.load(file)

with open("C:\\Users\\arik_\\Documents\\Dokumente\\Job_Clausthal\\TNTM\\TNTM_Revision_TNNLS\\TNTM\Data\DataOctis2\\octis_dataset_20ng.pickle", "rb") as file:
    octis_dataset = pickle.load(file)



In [21]:
from Code.Evaluate.Metrics import score_all, get_tw_embeddings


with open("C:\\Users\\arik_\\Documents\\Dokumente\\Job_Clausthal\\TNTM\\TNTM_Revision_TNNLS\\TNTM\Data\DataOctis\\cleaned_embedding_df_20ng_BERT.pickle", 'rb') as f:
    embedding_df = pickle.load(f)


100%|██████████| 48018/48018 [12:04<00:00, 66.27it/s]


In [22]:
corpus = octis_dataset.get_corpus()
vocab = octis_dataset.get_vocabulary()

In [23]:
len(corpus), len(vocab)

(18846, 3350)

In [24]:
corpus_red = corpus[:100]
vocab_red = [word for doc in corpus_red for word in doc]
vocab_red = list(set(vocab_red))

In [25]:
octis_dataset._corpus = corpus_red
octis_dataset._vocabulary = vocab_red

In [26]:
def data2params_prodlda(corpus, vocab):
    return {
        "vocab_size": len(vocab),
    }



In [27]:
model = TopMost2OctisAdapter(
    model_topmost = ProdLDA,
    model_kwargs= {
        "num_topics": 10
    },
    data2_additional_kwargs = data2params_prodlda,
    batch_size = 512
)

In [28]:
len(vocab_red)

2449

In [29]:
res = model.fit(octis_dataset)

loading word embeddings: 100%|██████████| 2449/2449 [00:00<00:00, 5919.49it/s]


2449


100%|██████████| 200/200 [00:04<00:00, 44.99it/s]


In [30]:
evaluation_result = score_all(
    dataset = octis_dataset,
    tw_emb=tw_emb,
    n_words=10,
    result = {'topics': res[0], 
              "topic-word-matrix": res[1]},
)

100%|██████████| 2/2 [00:26<00:00, 13.01s/it]


In [31]:
evaluation_result

{'NPMI': 0.3165717173853347, 'WE_CO_PW': 0.21}

In [32]:
from TNTM.Benchmark.Benchmark import Benchmark

In [37]:
bench = Benchmark(
    octis_dataset = octis_dataset,
    embedding_df = embedding_df,
    models = [ProdLDA],
    model_specific_data2params_fun_list = [data2params_prodlda],
    n_topics = [10, 15],
    batch_size = 256
)

In [38]:
r = bench.run()

loading word embeddings: 100%|██████████| 2449/2449 [00:00<00:00, 3698.61it/s]


2449


100%|██████████| 2/2 [00:32<00:00, 16.30s/it]


Done with model <class 'topmost.models.basic.ProdLDA.ProdLDA'> and n_topics 10


loading word embeddings: 100%|██████████| 2449/2449 [00:00<00:00, 3956.46it/s]


2449


100%|██████████| 1/1 [44:43<00:00, 2683.25s/it]

Done with model <class 'topmost.models.basic.ProdLDA.ProdLDA'> and n_topics 15


In [39]:
r

[{'NPMI': 0.29702960186308736, 'WE_CO_PW': 0.21},
 {'NPMI': 0.3185537209523118, 'WE_CO_PW': 0.16}]